나스닥 마3룰 간단한 BT 위한 코드

- 마3룰 정의 (현 코드 내, 실제와 약간의 차이가 있을 수 있음)

    기본적으로  나스닥 종합주가지수 Buy and Hold

1. 나스닥 종합주가지수가 3% 이상 하락하면, 그 날 종가에 나스닥 종합주가지수 판매, 최근 21 거래일 중 3% 하락일 있으면 재진입 X
2. 최근 21 거래일 중 나스닥 종합주가지수가 3% 이상 하락한 케이스가 4회 이상이면, 42 거래일간 나스닥 종합주가지수 재진입 X

In [3]:
import pandas as pd

Import Data

In [4]:
df = pd.read_csv("^IXIC.csv")

In [5]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1971-02-05,100.000000,100.000000,100.000000,100.000000,100.000000,0
1,1971-02-08,100.839996,100.839996,100.839996,100.839996,100.839996,0
2,1971-02-09,100.760002,100.760002,100.760002,100.760002,100.760002,0
3,1971-02-10,100.690002,100.690002,100.690002,100.690002,100.690002,0
4,1971-02-11,101.449997,101.449997,101.449997,101.449997,101.449997,0


In [6]:
df = df.sort_values(["Date"]).reset_index(drop=True)

### 1. 나스닥 마3 이후 21거래일 간의 리턴 확인

마3 이후 리턴이 낮기 때문에 홀딩하지 말라는 것일지.

Calculate Ex-ante Return, Ex-post Return

In [7]:
df["RET"] = df["Close"] / df["Close"].shift(1) #Date 날 기준의 RET

df["RET_POST21"] = df["Close"].shift(-21)/df["Close"] #Date 날 종가에 구매하고 21일 이후에 판매하였을 때 리턴
df["RET_POST21_GAP1"] = df["Close"].shift(-22)/df["Close"].shift(-1) #Date의 다음 날 종가에 구매하고 22일 이후에 판매하였을 때 리턴

In [8]:
df1 = df[["Date","RET","RET_POST21","RET_POST21_GAP1"]].dropna().reset_index(drop=True)

In [9]:
df1.corr()

C:\Users\744id\AppData\Local\Temp\ipykernel_8992\473017434.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df1.corr()


,RET,RET_POST21,RET_POST21_GAP1
RET,1.000000,0.021885,0.013881
RET_POST21,0.021885,1.000000,0.957034
RET_POST21_GAP1,0.013881,0.957034,1.000000


In [11]:
df1[df1["RET"]<0.97].describe()

,RET,RET_POST21,RET_POST21_GAP1
count,253.000000,253.000000,253.000000
mean,0.957660,1.010203,1.009596
std,0.014699,0.109009,0.108255
min,0.876787,0.765835,0.737502
25%,0.955101,0.936182,0.942688
50%,0.962093,1.010769,1.010108
75%,0.967106,1.090228,1.082083
max,0.969924,1.282845,1.337277


In [12]:
df1[df1["RET"]>=0.97].describe()

,RET,RET_POST21,RET_POST21_GAP1
count,12941.000000,12941.000000,12941.000000
mean,1.001287,1.009704,1.009717
std,0.011107,0.059731,0.059758
min,0.970015,0.660109,0.660109
25%,0.995870,0.977963,0.977954
50%,1.001227,1.014365,1.014386
75%,1.006321,1.045385,1.045475
max,1.141732,1.337277,1.278087


1] sample 수 차이가 있고, 2] std 차이가 있기 때문에 

통계적인 테스트를 할 때는 주의가 필요해보임.

하지만, 통계적인 테스트를 하지 않더라도, 나스닥 마3이 발생하였을 때 이후 21일 리턴 (1.02%)은, 그렇지 않은 이후 21일 리턴 (0.97%)에 비해 오히려 더 높음.

하루의 갭을 두고 보더라도 유의미한 차이가 나지 않음 (매매 수수료 등을 감안해 보았을 때)

즉 1971년 - 2023년 나스닥 데이터 기준으로, 지수 마3이 발생하였을 때 이후 21거래일 수익률이 낮다는 근거는 제시할 수 없음

### 2. 나스닥 마3 이후에 21일간 마3이 없을 때 까지 구매하지 않는 전략 BT

이를 다시 해석하면, 최근 21거래일간 수익률의 최저값이 -3%보다 낮으면 Hold 하지 않고, -3%보다 높으면 Hold 하는 것과 동일함.

In [13]:
df["RET_MIN_ROL21"] = df["RET"].rolling(21).min().shift(1) #전날 종가까지의 리턴 기준으로 판단해야 하므로.
df.loc[df["RET_MIN_ROL21"]<0.97, "ifHold"] = 0 #과거 21개 거래일 중 나스닥 리턴이 -3이 발생하면, 홀딩을 하지 않는다.
df.loc[df["RET_MIN_ROL21"]>=0.97, "ifHold"] = 1 #과거 21개 거래일 중 나스닥 리턴이 -3이 없으면, 홀딩 한다.
df.groupby(["ifHold"])["RET"].agg(["mean","std","prod","count"])
#이 케이스에는 확실히 마3이 있는 케이스가 마3이 없는 케이스보다 mean이 높고, std가 낮음.
#여전히 mean이 1보다 크긴 하지만... 심지어 누적 리턴은 1보다 낮음.

,mean,std,prod,count
ifHold,,,,
0.0,1.000123,0.021975,0.735279,2590
1.0,1.000531,0.009077,179.647796,10605


How about 마2?

In [14]:
df.loc[df["RET_MIN_ROL21"]<0.98, "ifHold_2"] = 0
df.loc[df["RET_MIN_ROL21"]>=0.98, "ifHold_2"] = 1
df.groupby(["ifHold_2"])["RET"].agg(["mean","std","prod","count"])
#마2로 가면 이게 조금 더 약해지지만 여전히 어느정도 차이는 남.

,mean,std,prod,count
ifHold_2,,,,
0.0,1.000288,0.017764,1.971215,5236
1.0,1.000558,0.007702,67.010079,7959


마4?

In [15]:
df.loc[df["RET_MIN_ROL21"]<0.96, "ifHold_4"] = 0
df.loc[df["RET_MIN_ROL21"]>=0.96, "ifHold_4"] = 1
df.groupby(["ifHold_4"])["RET"].agg(["mean","std","prod","count"])
#마4로 가면 평균의 차이는 얼마 없지만, 이 기간에 hold 하지 않는게 괜찮은 전략으로 보임.

,mean,std,prod,count
ifHold_4,,,,
0.0,1.000363,0.028499,0.959867,966
1.0,1.000458,0.010470,137.614185,12229


결론: 과거 데이터는 마3룰 중 1번의 결과와 일관됨 

최근 21거래일 내에 나스닥 마3이 발생하였을 때, 나스닥을 홀딩한다면 cumulative return은 2590거래일 기준 0.7352로 1보다 낮음

Daily 평균 RET의 경우에도 21거래일 내에 마3이 발생했을 때가, 마3이 발생하지 않았을 때에 비해 더 낮음

future work:

BT의 결과가 미래에도 그대로 발생한단 가정을 하기 위해서는

과거 분포가 현재 분포와 같다는 가정이 있어야 하며

또한 mean을 볼지 prod를 볼지 결정하기 위해서는

RET가 normal distribution을 따르는지, 아니면 log normal distribution을 따르는지에 대한 차이를 생각해 봐야 할 것으로 보임.

normal distribution을 따른다는 가정 하에서는, 

최근 21거래일에 마3이 발생하였어도 평균 수익률은 +기 때문에, expected future return이 +지만 (mean median 모두)

log normal distribution을 따른다는 가정 하에서는,

최근 21거래일에 마3이 발생하였을 때 log(RET)의 평균은 -기 때문에, future return 분포의 median이 -임. (mean은 체크해봐야함)

### 3. 최근 21거래일간 나스닥 종합주가지수가 3% 하락한 날이 4거래일 이상일 때 42 거래일간 홀딩하지 않는 전략 BT

이를 코드로 구현하면, 최근 42거래일간, 최근 21거래일의 마3횟수 >= 4 인 날짜가 있을 때 홀딩하지 않는 것으로 해석할 수 있음

In [16]:
df["minusThree"] = 0
df.loc[df["RET"]<0.97, "minusThree"] = 1
df["numberOfMinusThreeIn21Days"] = df["minusThree"].rolling(21).sum().shift(1)
df.loc[df["numberOfMinusThreeIn21Days"].rolling(42).max()>=4, "ifHoldRuleTwo"] = 0
df.loc[df["numberOfMinusThreeIn21Days"].rolling(42).max()<4, "ifHoldRuleTwo"] = 1


In [17]:
df.groupby(["ifHoldRuleTwo"])["RET"].agg(["mean","std","prod","count"])
#Rule 1에 비해 count 수는 적지만, 역시 평균 리턴도 더 낮고 누적 리턴도 더 낮음.

,mean,std,prod,count
ifHoldRuleTwo,,,,
0.0,1.000267,0.029067,0.874054,869
1.0,1.000460,0.010638,141.287048,12286


In [18]:
df.groupby(["ifHold","ifHoldRuleTwo"])["RET"].agg(["mean","std","prod","count"])
#아까의 Rule과 같이 보았을 때는, 굳이 Rule Two가 필요한지 의문이 들긴 함. 

mean       std        prod  count
ifHold ifHoldRuleTwo                                       
0.0    0.0            1.000097  0.029583    0.761423    803
       1.0            1.000134  0.017523    0.965665   1787
1.0    0.0            1.002328  0.021905    1.147922     66
       1.0            1.000515  0.008954  146.310685  10499

### 4. 결론

현재 분석에 사용한 period 기준으로 과거 21거래일에 마3이 발생하였을 때 나스닥을 홀딩하면 손해를 봤던 것으로 보임.

하지만 In-Sample 결과기도 하고, 몇몇 분석이 충돌이 발생하는 등 현재 이 것을 내 투자 전략에 반영하기에는 무리가 있어보임.

Rough하게 확인해 본 결과 마3룰의 1번을 더 분석해 볼 여지는 있어보임.

Future Work: 
- Nasdaq 이외의 시장에서도 확인해보기
- 왜 마3룰 관련 21일 홀딩 결과와 누적 리턴 결과가 일관되지 않은지 확인해보기
    - 마3이 발생하였을 때 std가 더 높기 때문에 prod가 더 낮은건지?
    - 아니면 21일 홀딩 기준으로 분석하였을 때는 겹치는 기간이 있기 때문?
    - 이러한 차이점에 대해 분석해 보면 어쩌면 V자 반등 시작을 잡는 좋은 trigger가 될 수도 있어보임.
- 미국 시장 타 지수로도 1926년부터 확인해보기